##### 对所有静态分析结果进行统计分析

In [1]:
import os
import json
import pprint

static_res_dir = '/root/minidroid/result/sensi_apis'
pp_dir = '/root/minidroid/dataset/privacy_policy'
all_res = {}
for file in os.listdir(static_res_dir):
    with open(os.path.join(static_res_dir, file), 'r') as fp:
        data = json.load(fp)
    all_res[file.replace('.json', '')] = data

miniset = set()
sensi_apis_count = {}
reachable_sensi_apis_count = {}
unreachable_sensi_apis_count = {}

for miniapp in all_res.keys():
    res = all_res[miniapp]
    for page in res.keys():
        apis = res[page]
        for api in apis.keys():
            if api not in sensi_apis_count.keys():
                sensi_apis_count[api] = 1
            else:
                sensi_apis_count[api] += 1
            if apis[api] is None:
                if api not in unreachable_sensi_apis_count.keys():
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] = 1
                else:
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] += 1
            else:
                if api not in reachable_sensi_apis_count.keys():
                    reachable_sensi_apis_count[api] = 1
                else:
                    reachable_sensi_apis_count[api] += 1

# pprint.pprint(sensi_apis_count)
# pprint.pprint(reachable_sensi_apis_count)
# pprint.pprint(unreachable_sensi_apis_count)

counts = 0
for count in unreachable_sensi_apis_count.values():
    counts += count
print(counts)

13172


##### 对所有静态分析结果进行一致性分析

In [15]:
import config as config

class ConsistencyAnalyzer():
    def __init__(self, appid, sensi_apis, pp_guidelines):
        self.appid = appid
        self.sensi_apis = sensi_apis
        self.pp_guidelines = pp_guidelines
        self.reachable_sensi_apis = set()
        self.unrechable_sensi_apis = set()
        self.reachable_scopes = set()
        self.pp_scopes = self.privacy_policy_to_scopes()
        self.set_reachable_sensi_apis()
        self.res = {
            'appid': self.appid,
            'app_nickname': self.pp_guidelines['app_nickname'],
            'reachable_sensi_apis': list(self.reachable_sensi_apis),
            'unreachable_sensi_apis': list(self.unrechable_sensi_apis),
            'pp_scopes': list(self.pp_scopes),
            'redundant_scopes': [],
            'missing_scopes': []
        }
        self.consistency_analysis()
    
    def set_reachable_sensi_apis(self):
        for page in self.sensi_apis.keys():
            for sensi_api in self.sensi_apis[page].keys():
                if self.sensi_apis[page][sensi_api] == None:
                    self.unrechable_sensi_apis.add(sensi_api)
                else:
                    self.reachable_sensi_apis.add(sensi_api)
                    self.reachable_scopes.add(config.SENSITIVE_API[sensi_api])

    def privacy_policy_to_scopes(self):
        pp_scopes = set()
        for desc in self.pp_guidelines['privacy_detail_list']['item']:
            if '开发者将在获取你的明示同意后' in desc:
                scope = desc.split('，')[-1]
                pp_scopes.add(scope)
            elif '用于' in desc:
                scope = desc.split('，')[0].replace('开发者', '')
                pp_scopes.add(scope)
        return pp_scopes
    
    def consistency_analysis(self):
        redundant_scopes = list(self.pp_scopes - self.reachable_scopes)
        missing_scopes = list(self.reachable_scopes - self.pp_scopes)
        if len(redundant_scopes):
            self.res['redundant_scopes'] = redundant_scopes
        else:
            self.res['redundant_scopes'] = None
        if len(missing_scopes):
            self.res['missing_scopes'] = missing_scopes
        else:
            self.res['missing_scopes'] = None


consistency_res = []
for file in os.listdir(static_res_dir):
    try:
        appid = file.replace('.json', '')
        with open(os.path.join(static_res_dir, file), 'r') as fp:
            sensi_apis = json.load(fp=fp)
        with open(os.path.join(pp_dir, file), 'r') as fp:
            pp_guidelines = json.load(fp=fp)
        analyzer = ConsistencyAnalyzer(appid=appid, sensi_apis=sensi_apis, pp_guidelines=pp_guidelines)
        res = analyzer.res
        if res['redundant_scopes'] != None:
            consistency_res.append(res)
    except Exception as e:
        pass

with open('/root/minidroid/result/redundant_scopes_res.json', 'w') as fp:
    json.dump(consistency_res, fp=fp, indent=4, ensure_ascii=False)
    